In [ ]:
!pip install efficientnet_pytorch
import numpy as np
import pandas as pd
import os
import json
import random
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from efficientnet_pytorch import EfficientNet
from albumentations import *
from albumentations.pytorch import ToTensorV2

random_state = 42

random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
torch.cuda.manual_seed(random_state)

I wanted to predict something using model with multiple outputs. This is a good dataset for this task.

In [ ]:
root = '/kaggle/input/weathertime-classification-with-road-images'

annotation = open(os.path.join(root,'train_dataset', 'train.json'))
data = json.load(annotation)
print('Total:', len(data['annotations']))

In [ ]:
train_ds = pd.json_normalize(data['annotations'])
train_ds.head()

In [ ]:
#period_encoder = LabelEncoder().fit(train_ds['period'])
weather_encoder = LabelEncoder().fit(train_ds['weather'])

#train_ds['period'] = period_encoder.transform(train_ds['period'])
train_ds['weather'] = weather_encoder.transform(train_ds['weather'])
train_ds['filename'] = train_ds['filename'].str.replace('\\', '/', regex=True)

#print(train_ds['period'].value_counts())
print(train_ds['weather'].value_counts())

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_ds['filename'].values, train_ds['weather'].values, 
                                                    shuffle=True, 
                                                    random_state=random_state, 
                                                    stratify=train_ds['weather'])

In [ ]:
transforms = {
    x: Compose([
        Resize(224, 224),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.2, border_mode=cv2.BORDER_REPLICATE),
        RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
        OneOf([
            GaussianBlur(),
            GaussNoise(),
        ], p=0.2),

        Normalize(),
        ToTensorV2()
    ]) if x == 'train' else Compose([
        Resize(224, 224),
        
        Normalize(),
        ToTensorV2()
    ]) for x in ['train', 'test']
}

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_files, labels, transform=None):
        self.image_files=image_files
        self.labels=labels
        self.transform=transform
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, i):
        image = np.array(Image.open(os.path.join(root, 'train_dataset', self.image_files[i])).convert('RGB'))
        labels = self.labels[i]
        
        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        return image, labels

In [ ]:
sets = {
    'train': (x_train, y_train), 'test': (x_test, y_test)
}
datasets = {
    x: CustomDataset(sets[x][0], sets[x][1], transforms[x]) for x in sets.keys() 
}
dataloaders = {
    x: DataLoader(datasets[x], batch_size=8, num_workers=2, pin_memory=True) for x in sets.keys()
}

In [ ]:
#model = EfficientNet.from_pretrained('efficientnet-b0')
model = torchvision.models.mobilenet_v3_small(pretrained=True)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)
lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = torch.nn.CrossEntropyLoss().to(device)
epochs = 10

In [ ]:
def train_fn(i, model, data_loader, loss_fn, optimizer, device):
    overall_loss = 0.
    
    with tqdm(data_loader, total=len(data_loader), desc=f'Training, phase {i} :') as loader:
        for data, weather in loader:
            optimizer.zero_grad()

            data, weather = data.to(device), weather.to(device)
            output = model(data)
            loss = loss_fn(output, weather)
            
            overall_loss += loss.item()

            loss.backward()
            optimizer.step()

            loader.set_postfix(loss=overall_loss / len(data_loader))
            

def validation_fn(i, model, data_loader, device):
    model.eval()
    
    weather_accuracy = 0.
    
    with tqdm(data_loader, total=len(data_loader), desc=f'Validating, phase {i} :') as loader:
        with torch.no_grad():
            for data, weather in loader:
                data, weather = data.to(device), weather.to(device)

                output = model(data)
                
                weather_accuracy += accuracy_score(weather.cpu().detach().numpy(), 
                                                   torch.argmax(output, dim=1).cpu().detach().numpy())

                loader.set_postfix(accuracy_weather=weather_accuracy / len(data_loader))
    model.train()

In [ ]:
torch.cuda.empty_cache()
for epoch in range(1, epochs + 1):
    train_fn(epoch, model, dataloaders['train'], loss_fn, optimizer, device)
    validation_fn(epoch, model, dataloaders['test'], device)

In [ ]:
model.eval()

images = glob.glob(os.path.join(root, 'test_dataset', 'test_images', '*'))

for _ in range(5):
    image = np.array(Image.open(np.random.choice(images)).convert('RGB'))
    print(image.shape)
    
    tensor = transforms['test'](image=image)['image'].unsqueeze(0).to(device)
    print(tensor.shape)
    output = model(tensor)
    
    output = torch.argmax(output, dim=1).cpu().detach().numpy()
    print(output)
    plt.imshow(image)
    plt.title(f'{weather_encoder.inverse_transform(output)[0]}')
    plt.axis('off')
    plt.show()

In [ ]:
filename = []
period = []
weather = []

for file in images:
    image = np.array(Image.open(file).convert('RGB'))
    
    tensor = transforms['test'](image=image)['image'].unsqueeze(0).to(device)
    output = model(tensor)
    
    output = torch.argmax(output, dim=1)

    filename.append(os.path.join(*file.rsplit('/', 2)[1:]).replace('/', '\\'))
    weather.append(output.item())

df = pd.DataFrame({'filename': filename,  
                   'weather': weather_encoder.inverse_transform(weather)}
                 ).sort_values('filename').reset_index(drop=True)

display(df)

In [ ]:
data = df.to_json(orient='records')
js_df = {"root": {"annotations": json.loads(data)}}
with open('submission.json', 'w') as out:
    json.dump(js_df, out, indent=4)

PS: I'm not very happy with the results. Might be because datasets are unbalanced. Also, the author said that dataset contains 5 weather classes and 5 period classes, but representation in train set is actually 3 and 4. 

I will keep this notebook here as an example. I will be very happy to hear healthy criticism and recommendations. 

If you liked it or found it useful plz UV ;) 

In [ ]:
torch.save(model.state_dict(), 'mobilenet_v3.pt')